# Práctica 12: Evaluación en clasificación..

## Realizado por:
- Susana Suárez Mendoza
- Mara Pareja del Pino

In [1]:
import pandas as pd
from sklearn import tree, model_selection, metrics, linear_model
import matplotlib.pyplot as plt
import numpy as np

### Ejercicio 1: Estimación del valor de las casas.

In [2]:
data_csv = pd.read_csv('./precio_casas.csv', sep = ';')

In [3]:
y = data_csv.iloc[:, -1].to_numpy()
X = data_csv.iloc[:, :-1].to_numpy()

1. Obtener el rendimiento de árbol de regresión mediante la técnica de división repetida (holdout
repetido), para los diferentes criterio de división de nodos que posee la clase.


In [4]:
rendimientos = []
criterios = ['squared_error', 'friedman_mse', 'absolute_error']
modelos = []

for i in range(1, 11):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=1/3)
    filas = []

    for criterio in criterios:
        reg_tree = tree.DecisionTreeRegressor(criterion=str(criterio))
        reg_tree.fit(X_train, y_train)
        y_pred = reg_tree.predict(X_test)
        filas.append(round(metrics.r2_score(y_test, y_pred)*100, 4))

    rendimientos.append(filas)

df_tree = pd.DataFrame(rendimientos, columns=criterios)
display(df_tree)

,squared_error,friedman_mse,absolute_error
0,58.3302,58.6380,55.8881
1,59.7645,58.9369,55.9698
2,60.0971,60.2620,57.9305
3,59.0905,59.0465,58.2271
4,61.8158,61.1426,58.0412
5,59.5716,59.6171,59.2589
6,61.1912,60.0320,59.2538
7,60.3699,60.4328,62.9664
8,58.9648,59.0426,57.6249
9,60.0568,59.6710,57.2756


2. Obtener el rendimiento de un modelo de regresión lineal mediante la técnica de división repetida (holdout repetido).


In [5]:
rendimientos = []
criterios = ['squared_error', 'friedman_mse', 'absolute_error']

for i in range(1, 11):
    filas = []
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=1/3)

    reg_lineal = linear_model.LinearRegression()
    reg_lineal.fit(X_train, y_train)
    y_pred = reg_lineal.predict(X_test)

    filas.append(round(metrics.r2_score(y_test, y_pred)*100, 4))
    filas.append(round(metrics.mean_squared_error(y_test, y_pred)*100, 4))
    filas.append(round(metrics.mean_absolute_error(y_test, y_pred)*100, 4))

    rendimientos.append(filas)

df_lineal = pd.DataFrame(rendimientos, columns=criterios)
display(df_lineal)

,squared_error,friedman_mse,absolute_error
0,23.5584,103.4672,54.0683
1,60.2464,52.2535,53.1571
2,35.3317,86.2662,53.8266
3,60.4843,53.0890,52.7446
4,60.3523,52.3125,53.2476
5,59.5961,52.0509,52.6904
6,59.0004,53.7054,53.0899
7,60.1254,52.0907,52.9486
8,58.9564,54.7711,53.1015
9,61.0994,51.5581,52.9641


3. Comparar los resultados anteriores en función del valor medio y la desviación típica.

In [6]:
mean_tree = df_tree['friedman_mse'].mean()
mean_lineal = df_lineal['friedman_mse'].mean()

std_tree = df_tree['friedman_mse'].std()
std_lineal = df_lineal['friedman_mse'].std()

df = pd.DataFrame({'Media': [mean_tree, mean_lineal], 'Desviación típica': [std_tree, std_lineal]}, index=['Árbol de regresión', 'Regresión lineal'])
display(df)

,Media,Desviación típica
Árbol de regresión,59.68215,0.790084
Regresión lineal,61.15646,18.247614


Se observa que ambos modelos tienen una media similar, sin embargo, aquel modelo cuya desviación típica es menor, y por lo tanto obtiene un mejor resultado es el de árbol de regresión

### Ejercicio 2: Implementación de la validación cruzada

In [7]:
def validacion_cv(n, n_particiones=5, mezclar=True, semilla=None):
    particiones = []
    indices = np.arange(n)

    if mezclar:
        np.random.seed(semilla)
        np.random.shuffle(indices)
   
    grupos = np.array_split(indices, n_particiones)

    for i in range(n_particiones):
        test = grupos[i]
        train = np.concatenate([grupos[j] for j in range(n_particiones) if j != i])

        tuple = (train, test)
        particiones.append(tuple)

    return particiones

In [8]:
indices = validacion_cv(len(X), n_particiones=5, semilla=12345)
rendimientos = []

for indice in indices:
    X_train_ind = indice[0]
    X_test_ind = indice[1]

    X_train, X_test, y_train, y_test = X[X_train_ind], X[X_test_ind], y[X_train_ind], y[X_test_ind]

    filas = []

    reg_tree = tree.DecisionTreeRegressor(criterion='squared_error')
    reg_tree.fit(X_train, y_train)
    y_pred = reg_tree.predict(X_test)
    filas.append(round(metrics.r2_score(y_test, y_pred)*100, 4))

    rendimientos.append(filas)

df_tree = pd.DataFrame(rendimientos, columns=['squared_error'])

mean = df_tree['squared_error'].mean()
std = df_tree['squared_error'].std()
print('Media: ', round(mean, 2))
print('Desviación típica: ', round(std, 2))


Media:  61.32
Desviación típica:  1.71


Finalmente tras usar Cross-Validation no observamos una diferencia significativa en los resultados, aunque al mejorar la media mejora el modelo.

In [12]:
model = tree.DecisionTreeRegressor(criterion='squared_error')
k_fold = model_selection.KFold(n_splits=5, shuffle=True, random_state=12345)
results = []
for train, test in k_fold.split(X):
    model.fit(X[train], y[train])
    y_pred = model.predict(X[test])
    results.append(round(metrics.r2_score(y[test], y_pred)*100, 4))

df = pd.DataFrame(results, columns=['squared_error'])

mean = df['squared_error'].mean()
std = df['squared_error'].std()

print('Media: ', round(mean, 2))
print('Desviación típica: ', round(std, 2))

Media:  61.73
Desviación típica:  1.23


Además, si realizamos el mismo proceso pero utilizando Cross-Validation de scikit-learn el resultado es bastante similar.